You can use this notebook to generate texts using Falcon.

In [ ]:
# clone Lit-GPT
!git clone https://github.com/Lightning-AI/lit-gpt
%cd lit-gpt/

Cloning into 'lit-gpt'...
remote: Enumerating objects: 1527, done.
remote: Counting objects: 100% (375/375), done.
remote: Compressing objects: 100% (125/125), done.
remote: Total 1527 (delta 310), reused 271 (delta 249), pack-reused 1152
Receiving objects: 100% (1527/1527), 495.45 KiB | 5.38 MiB/s, done.
Resolving deltas: 100% (996/996), done.


In [ ]:
# install the dependencies
!pip install -r requirements-all.txt -q

In [ ]:
# download the weights
!python scripts/download.py --repo_id tiiuae/falcon-7b
!python scripts/convert_hf_checkpoint.py --checkpoint_dir checkpoints/tiiuae/falcon-7b

In [ ]:
# run inference
!python generate/base.py \
        --prompt "Hello, my name is" \
        --checkpoint_dir checkpoints/tiiuae/falcon-7b \
        --quantize bnb.int8

Time to load model: 148.99 seconds.
Global seed set to 1234
Hello, my name is Jack.
Some people think that dogs are like people. They see them as loyal, loving, and caring friends. I don't believe that's true. Dogs are animals. Animals are animals. I'm not a dog person.
Time for inference 1: 7.82 sec total, 6.39 tokens/sec
Memory used: 10.11 GB
